In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: c:\Users\schue\Git_SSD\FlareSense


In [2]:
import os

os.getcwd()

'c:\\Users\\schue\\Git_SSD\\FlareSense'

In [3]:
import torch
import mlflow
import dagshub
import torchmetrics
import src.utils.data as data
import pytorch_lightning as pl

from torchvision import transforms
from tqdm.notebook import tqdm
from src.models.ResNet50BinaryClassifier import ResNet50BinaryClassifier
from src.models.EfficientNetv2sBinaryClassifier import EfficientNetV2SBinaryClassifier

mlflow.pytorch.autolog()
torch.set_float32_matmul_precision("high")

## Training

Defining the data and model

In [4]:
model = EfficientNetV2SBinaryClassifier(lr=1e-3, weight_decay=1e-4)
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=64,
    num_workers=16,
    val_ratio=0.15,
    test_ratio=0.15,
    split_by_date=True,
    filter_instruments=["australia_assa_02"],
)
data_module.setup()

Training and logging the model

In [5]:
dagshub.init("FlareSense", "FlareSense", mlflow=True)
mlflow.start_run()

run_id = mlflow.active_run().info.run_id
print(f"Run ID: {run_id}")
print(f"Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_{run_id}")

trainer = pl.Trainer(max_epochs=2, log_every_n_steps=1)

trainer.fit(
    model,
    train_dataloaders=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader(),
)

trainer.test(model, dataloaders=data_module.test_dataloader())

mlflow.end_run()

Repository initialized!

Run ID: c1817cb2851f45ee85afb02343157095
Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_c1817cb2851f45ee85afb02343157095


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | efficientnet_v2_s | EfficientNet    | 20.2 M
1 | precision         | BinaryPrecision | 0     
2 | recall            | BinaryRecall    | 0     
------------------------------------------------------
18.4 M    Trainable params
1.8 M     Non-trainable params
20.2 M    Total params
80.715    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
2023/11/10 17:28:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\schue\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.07510808110237122    │
│      test_precision       │    0.9551281929016113     │
│        test_recall        │     0.764102578163147     │
└───────────────────────────┴───────────────────────────┘

## Testing

Downloading the model from mlflow and putting it on the correct device

In [5]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

mlflow.set_tracking_uri('https://dagshub.com/FlareSense/Flaresense.mlflow')
model = mlflow.pytorch.load_model(f"runs:/{run_id}/model/").to(device)
model.eval()

ResNet50BinaryClassifier(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequ

Run through testdata

In [6]:
test_labels_list = []
test_preds_list = []
with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        images, info = batch
        images = images.to(device)
        binary_labels = [0 if label == "no_burst" else 1 for label in info['label']]
        binary_labels = torch.tensor(binary_labels).int().view(-1, 1)
        binary_labels = binary_labels.to(device)
        
        outputs = model(images.expand(-1, 3, -1, -1))
        predictions = (outputs >= 0.5).int()
        
        test_labels_list.append(binary_labels)
        test_preds_list.append(predictions)

  0%|          | 0/68 [00:00<?, ?it/s]

Create confmatrix

In [7]:
test_labels = torch.cat(test_labels_list, dim=0)
test_preds = torch.cat(test_preds_list, dim=0)

confmat_metric = torchmetrics.ConfusionMatrix(num_classes=2, task="binary").to(device)
confmat_metric(test_preds, test_labels)

tensor([[3973,   17],
        [  24,  321]], device='cuda:0')